## This notebook may be used to reproduce generate data for Figure 6 in the main paper. (The figure is plotted in another notebook:  `SimExampleFig2Fig4Fig6.ipynb`)

### Parameter values that need to be changed to derive the different figures are mentioned in comments as appropriate. Note that nonlinear and linear versions of the interneuronal inhibition model can be chosen.

In [3]:
import ipyparallel as ipp
clients = ipp.Client()

In [4]:
clients.ids

[0, 1, 2, 3]

In [5]:
dview = clients.direct_view()

In [6]:
with dview.sync_imports():
    import numpy as np
    from math import exp,sqrt,log
    import numpy.random
%px np = numpy
%px rand = numpy.random

importing numpy on engine(s)
importing exp,sqrt,log from math on engine(s)
importing numpy.random on engine(s)


In [7]:
def Sim_Anim_Beh(trials,randSeed,rin):
    
    rand.seed(randSeed)
    
    ##############################################################################################
    # Runge_Kutta 4th-order
    ##############################################################################################
    def rk4(F,t,y,ht):
        K0 = ht*F(t,y,dfct)
        K1 = ht*F(t + ht/2.0, y + K0/2.0,dfct)
        K2 = ht*F(t + ht/2.0, y + K1/2.0,dfct)
        K3 = ht*F(t + ht, y + K2,dfct)
        return (K0 + 2.0*K1 + 2.0*K2 + K3)/6.0

    ####################################################
    # Predictor-corrector integration routine: Heun for stochastic differential equations
    ####################################################
    # time t
    # time step ht
    # number of equations (=dimensionality) n
    # random numbers xi
    # deterministic and stochastic contributons Fdet and Frand
    def Heun(Fdet, Frand,t,y,ht):
        fd1 = np.zeros(3)
        fd2 = np.zeros(3)
        fr = np.zeros(3)
        yt = np.zeros(3)
        fd1 = Fdet(t,y,dfct)
        fr = Frand()
        yt = y + ht*fd1 + fr*sqrt(ht)
        fd2 = Fdet(t+ht,yt,dfct)
        ht2 = ht/2.0
        return ht2*(fd1+fd2) + fr*sqrt(ht)
    
    #activation function
    def actfct(z,g,b):
        return 1/(1+exp(-g*(z-b)))

    #differential equations motivations
    #determnistic part
    
    
    #nonlinear model
    #use this model for Figs 2A-F and Figs 4B and 4C (and comment linear model below)
    def Fdet(t,y,dfct):
        Fdet = np.zeros(3)
        Fdet[0] = -k*y[0] + beta*r*actfct(y[0],g1,b1) - beta*actfct(y[2],g2,b2) + q*dfct[0]
        Fdet[1] = -k*y[1] + beta*r*actfct(y[1],g1,b1) - beta*actfct(y[2],g2,b2) + q*dfct[1]
        Fdet[2] = -kinh*y[2] + w_exc*(actfct(y[0],g1,b1) + actfct(y[1],g1,b1))
        return Fdet
    
    '''  
    #linear model
    # use this model for Figs 4E and F (and comment nonlinear model above)
    def Fdet(t,y,dfct):
        Fdet = np.zeros(3)
        Fdet[0] = (r*beta*g1/4-k)*y[0] - beta*g2/4*y[2] + q*dfct[0] + phi
        Fdet[1] = (r*beta*g1/4-k)*y[1] - beta*g2/4*y[2] + q*dfct[1] + phi
        Fdet[2] = -kinh*y[2] + w_exc*(1 - g1*b1/2 + g1/4*(y[0] + y[1]))
        return Fdet
    '''
    
    def FrandDet():
        FrandDet = np.zeros(3)
        return FrandDet
    
    
    
    # stochastic part of RHS   
    def Frand():
        Frand = np.zeros(3)
        xRand0 = rand.normal(0,1)
        xRand1 = rand.normal(0,1)
        Frand[0] = sigma*xRand0
        Frand[1] = sigma*xRand1
        Frand[2] = 0
        return Frand
    
    # model parameters
    k = 0.8 # leak excitatory unit
    kinh = 0.8 # leak inhibitory unit
    w_exc = 3 # excitation strength inhibitory unit
    q = 0.1 # frequency of integration
    g1 = 10 # gain excitation function
    g2 = 10 # gain inhibition function
    b1 = 0.5 # midpoint excitation function
    b2 = 0.5 # midpoint inhibition function
    beta = 3 #3 # inhibition strength
    r = rin
    
    #phi for linear model
    phi = 0.5*beta*(r*(1 - g1*b1/2) - (1 - g2*b2/2))
    
    # geometric distribution for bout times
    # probability of interruption
    # calculation of maximum terminal bout time 'bout_max_99' including 99 percent of all values
    lam_interrupt = 0.05
    bout_max_99 = int(log(0.01)/log(1-lam_interrupt)) + 1
    
    eps_offset = 1e-9
    decay = 0.15
    
    ## for Fig2 (nonlinear model) choose:
    d01 = 7.5
    ## for Fig 4B (nonlinear model) choose:
    #d01 = 8.0
    ## for Fig 4C (nonlinear model), 4E (linear model) choose:
    #d01 = 8.1
    ## for Fig 4F (linear model) choose:
    #d01 = 8.35
    
    d02 = np.sqrt(2*7.5*7.5-d01**2) 
    #dfct_start = 7.5
    t = 0.0  #start time
    tend = 200  #terminal time 
    y = np.array([0.0, 0.0, 0.0])  #initial conditions
    dfct = np.array([d01, d02])
    ht = 0.005  #time step
    sigma = 0
    time1 = []  
    Ysol1 = []
    time1.append(t)
    Ysol1.append(y)

    while t <= tend:
        #ht = min(ht,tend-t)
        #y = y + Heun(Fdet,FrandDet,t,y,%%html)
        y = y + rk4(Fdet,t,y,ht)
        for ii in range(len(y)):
            y[ii] = max(0, y[ii])
        t = t + ht
        time1.append(t)
        Ysol1.append(y)
    
    y_init = y
    dfct_init = dfct
        
    Ep_sum_av = 0
    time2_list = []
    Ysol2_list = []
    Deficit_list = []
    All_tchange_Lists = []
    
    for nn in range(trials):
        tau_dist = 4.0 #time to overcome distance between food and water sources
        t = 0.0
        tau = tau_dist/2.0 #initially animal is in between the two sources
        tchange = 0.0
        sigma = 0.01
        nfood = 2
        nwater = 2

        Tep = []
        Ep = []
        Ep_sum = 0
        time2 = []  
        Ysol2 = []
        Deficit = []
        time2.append(t)
        Ysol2.append(y)
        Deficit.append(dfct)
        Tbout_max = bout_max_99
        tchange_list=[]
        
        tend = Tbout_max + 1
        while t <= tend:
            for kk in range(1,Tbout_max+1):
                if t > kk-ht/2 and t < kk+ht/2:
                    Tep.append(int(t+ht))
                    penalty = dfct[0]*dfct[0] + dfct[1]*dfct[1]
                    Ep.append(penalty*lam_interrupt*pow(1-lam_interrupt,int(t+ht)-1))
            
            
            if y[0] == y[1]:
                randNr = rand.uniform(0,1)
                if randNr < 0.5:
                    y[0] = y[0] + eps_offset
                else:
                    y[1] = y[1] + eps_offset
                    
            dely1 = y[0] - y[1]
            
            if y[0] > y[1]:
                if t > tchange + tau: 
                    dfct[0] = dfct[0] - decay*ht  
                    nfood = 1
                    nwater = 0
            elif y[1] > y[0]:
                if t > tchange + tau: 
                    dfct[1] = dfct[1] - decay*ht 
                    nwater = 1
                    nfood = 0   
                
            
            for jj in range(len(dfct)):
                dfct[jj] = max(0, dfct[jj])
            
            y = y + Heun(Fdet,Frand,t,y,ht)
            for ii in range(len(y)):
                y[ii] = max(0, y[ii])
                
            if y[0] == y[1]:
                randNr = rand.uniform(0,1)
                if randNr < 0.5:
                    y[0] = y[0] + eps_offset
                else:
                    y[1] = y[1] + eps_offset
                    
            dely2 = y[0] - y[1]
            
            if np.sign(dely1)*np.sign(dely2) < 0:
                tchange_old = tchange
                tchange = t
                
                tchange_list.append(tchange)
                if nfood == 1 or nwater == 1:
                    tau = tau_dist
                    nfood = 0
                    nwater = 0
                else:
                    tau = tau_dist-tau + tchange-tchange_old
            
            t = t + ht
            dfct = np.array([dfct[0], dfct[1]])
            Deficit.append(dfct)
            time2.append(t)
            Ysol2.append(y)
        
        time2_list = time2_list + time2
        Ysol2_list = Ysol2_list + Ysol2
        Deficit_list = Deficit_list + Deficit
        Ep_sum = np.sum(Ep)
        Ep_sum_av += Ep_sum/trials
        y = y_init
        dfct = dfct_init
        All_tchange_Lists.append(tchange_list)
    
    time2_av = []
    Ysol2_av = []
    Deficit_av = []
    for kk in range(len(time2)):
        time2_av.append(np.sum(time2_list[kk::len(time2)])/trials)
        Ysol2_av.append(sum(Ysol2_list[kk::len(time2)])/trials)
        Deficit_av.append(sum(Deficit_list[kk::len(time2)])/trials)
    return Ep_sum_av,time1,Ysol1,time2_av,Ysol2_av,Deficit_av,tau_dist,All_tchange_Lists

In [6]:
rlist = [1,1,2,2]
#rlist = [1,1,1,1]

import timeit
time_start = timeit.default_timer()
nr_engines = len(clients.ids)
randSeedList = [(jj+1)*1928374 for jj in range(nr_engines)]
nr_trials_tot = 2000
nr_trials_per_engine = int(nr_trials_tot/nr_engines)
results = dview.map_sync(Sim_Anim_Beh, [nr_trials_per_engine]*nr_engines, randSeedList, rlist)
time_stop = timeit.default_timer()
print('elapsed time =',time_stop - time_start,'seconds')
#print(randSeedList)

elapsed time = 5650.742929726141 seconds


In [7]:
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.lines import Line2D
import csv

In [17]:
diffThresh=4
results_r1_combined=results[0][7]+results[1][7]
with open("motivChanges_diffThresh"+str(diffThresh)+"_r_eq_1.csv", "w") as outf1:
    writer = csv.writer(outf1)
    writer.writerows(results_r1_combined)
    
results_r1=[np.asarray([y - x for x,y in zip(results_r1_combined[jj],results_r1_combined[jj][1:])]) for jj in range(len(results_r1_combined))]
results_r1_mod=[results_r1[jj][results_r1[jj]>diffThresh] for jj in range(len(results_r1))]
results_r1_len=[len(results_r1_mod[jj]) for jj in range(len(results_r1_mod))]
results_r1_maxLen = np.max(results_r1_len)
results_r1_minLen = np.min(results_r1_len)
print(results_r1_maxLen)
print(results_r1_minLen)
results_r1_mod=np.asarray([(list(results_r1_mod[jj]) + results_r1_maxLen * [0])[:results_r1_maxLen] for jj in range(len(results_r1_mod))])
results_r1_av=np.mean(results_r1_mod, axis=0)
print(results_r1_av)

results_r2_combined=results[2][7]+results[3][7]
with open("motivChanges_diffThresh"+str(diffThresh)+"_r_eq_2.csv", "w") as outf2:
    writer = csv.writer(outf2)
    writer.writerows(results_r2_combined)
results_r2=[np.asarray([y - x for x,y in zip(results_r2_combined[jj],results_r2_combined[jj][1:])]) for jj in range(len(results_r2_combined))]
results_r2_mod=[results_r2[jj][results_r2[jj]>diffThresh] for jj in range(len(results_r2))]
results_r2_len=[len(results_r2_mod[jj]) for jj in range(len(results_r2_mod))]
results_r2_maxLen = np.max(results_r2_len)
results_r2_minLen = np.min(results_r2_len)
print(results_r2_maxLen)
print(results_r2_minLen)
results_r2_mod=np.asarray([(list(results_r2_mod[jj]) + results_r2_maxLen * [0])[:results_r2_maxLen] for jj in range(len(results_r2_mod))])
results_r2_av=np.mean(results_r2_mod, axis=0)
print(results_r2_av)

11
5
[  9.559195   8.13731    8.68519    8.94972    9.82792   10.91148   10.66614
   7.208285   2.4729     0.386675   0.01292 ]
13
1
[  5.66567500e+00   5.86499000e+00   5.77306500e+00   5.62717500e+00
   5.36028000e+00   4.90903000e+00   4.15749500e+00   3.05442500e+00
   1.87500000e+00   9.05535000e-01   2.77485000e-01   3.51500000e-02
   5.49500000e-03]


In [ ]:
##The following is relevant for an extra figure in the APPENDIX

figExtra = plt.figure(figsize=(8,6))

plt.plot(np.arange(len(results_r1_av))+1,results_r1_av)
plt.plot(np.arange(len(results_r2_av))+1,results_r2_av)
#plt.plot(np.arange(len(results_r1_av[:results_r1_minLen]))+1,results_r1_av[:results_r1_minLen])
#plt.plot(np.arange(len(results_r2_av[:results_r2_minLen]))+1,results_r2_av[:results_r2_minLen])

plt.tight_layout();